# 사전 준비

**KcELECTRA 모델과 토크나이저 불러오기**

In [ ]:
!pip install transformers

In [2]:
from transformers import AutoTokenizer, AutoModel
  
tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
model = AutoModel.from_pretrained("beomi/KcELECTRA-base")

Downloading:   0%|          | 0.00/288 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/504 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/396k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraModel: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# 텍스트 전처리

**전처리 함수 정의**

In [ ]:
!pip install soynlp
!pip install emoji==1.7.0

In [4]:
import re
import emoji
from soynlp.normalizer import repeat_normalize

In [5]:
emojis = ''.join(emoji.UNICODE_EMOJI.keys())
pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣{emojis}]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

In [6]:
def clean(x):
    x = pattern.sub(' ', x)         # 일반적으로 사용하는 특수문자, 영어, 한글, emoji제외 공백으로 치환
    x = url_pattern.sub('', x)      # URL 제거
    x = x.strip()                   # 문자의 시작과 끝에서 공백제거
    x = repeat_normalize(x, num_repeats=2)      # 반목되는 문자의 축약 횟수 2개로 줄임
    return x

**데이터 불러오기**

In [7]:
import numpy as np
import pandas as pd

In [8]:
curse_data = pd.read_table("/content/drive/Othercomputers/내 컴퓨터/Curse-words_Detection/Curse-detection-data/dataset.txt", names=["text", "label"], sep="|", header=None)
curse_data

,text,label
0,좌배 까는건 ㅇㅂ,1
1,집에 롱 패딩만 세 개다. 10년 더 입어야지 ㅋㅋ,0
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,1
3,세탁이라고 봐도 된다,0
4,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,1
...,...,...
5819,좌우 헬파이어 3개씩 6개 장착에 아파치보다 약하지만 20mm 기관포 장착임,0
5820,"세금 내놓으라고 데모질 중 ㅋㅋ간첩, 도둑놈 새끼들이 대통령 해처먹으니까 나도 같...",1
5821,너가 한 말 중에,0
5822,제갈대중 ㅇㅂ,0


**전처리 및 데이터 분할**

In [19]:
# train : validation : test = 5 : 1 : 2

train_text = [clean(curse_data['text'][idx]) for idx in range(0, int((curse_data.shape[0]/8)*5))]
val_text = [clean(curse_data['text'][idx]) for idx in range(int((curse_data.shape[0]/8)*5), int((curse_data.shape[0]/8)*6))]
test_text = [clean(curse_data['text'][idx]) for idx in range(int((curse_data.shape[0]/8)*6), int((curse_data.shape[0]/8)*8))]

In [26]:
train_label = [curse_data['label'][idx] for idx in range(0, int((curse_data.shape[0]/8)*5))]
val_label = [curse_data['label'][idx] for idx in range(int((curse_data.shape[0]/8)*5), int((curse_data.shape[0]/8)*6))]
test_label = [curse_data['label'][idx] for idx in range(int((curse_data.shape[0]/8)*6), int((curse_data.shape[0]/8)*8))]

In [29]:
print("전처리 전:", curse_data['text'][10])
print("전처리 후:", train_text[10])

전처리 전: ㅋㅋㅋㅋㅋ
전처리 후: ㅋㅋ


**토크나이징**

In [30]:
train_input_token = tokenizer(train_text, truncation=True, padding=True, return_tensors="pt")
val_input_token = tokenizer(val_text, truncation=True, padding=True, return_tensors="pt")
test_input_token = tokenizer(test_text, truncation=True, padding=True, return_tensors="pt")

# 데이터 구축

**데이터셋**

In [33]:
import torch

In [34]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [35]:
train_dataset = MyDataset(train_input_token, train_label)
val_dataset = MyDataset(val_input_token, val_label)
test_dataset = MyDataset(test_input_token, test_label)

In [36]:
for n in range(3):
    print("train_dataset[",n,"]")
    print(train_dataset[n])

train_dataset[ 0 ]
{'input_ids': tensor([    2,  2896,  4225, 14257,  4030,   116,  4770,     3,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,   

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
